# Rehder on Daimler Implementation
---
> Aryan Garg  
> Dr. Amit S. Unde  
> Dr. Renu M. Rameshan  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import time
import math
import scipy.stats as stats
from scipy import signal
import random


import os
from os import listdir
from os.path import isfile, join
import sys

from google.colab.patches import cv2_imshow
from google.colab import files
from PIL import Image

import cv2
import torch
import torchvision
from torchvision import datasets, models, transforms
import torchvision.transforms as T

In [ ]:
class preProcessData:
    def readData(self, trainDataP):
        try:
            self.dirs = [f for f in listdir(trainDataP)]
            for directory in self.dirs:
                newPath = self.trainDataPath + directory + "/RectGrabber/"
                files = [f for f in listdir(newPath) if isfile(join(newPath, f))]
                self.allTrainingData[directory] = files
            #print(self.allTrainingData)
            return True
        
        except Exception as err:
            print(err)
            print("Couldn't read data. Check file paths and file health!")
            return False


    def showFrames(self, windowName, imglst):
        if len(windowName) != len(imglst):
            print(f"windowName list len: {len(windowName)} not equal to imglst len: {len(imglst)}")
            return False 

        for i in range(len(windowName)):
            cv2_imshow(imglst[i])
        
        k = cv2.waitKey(0)
        if k is not None:
            cv2.destroyAllWindows()
        
        return True
        
    def showSampleImages(self, folderName = "2012-06-05_165931"):
        ### TODO: Decrease Image size
        print("[+] Logging sample images' details\n---------------------")
        for i in range(1,10,2): 
            img_left = self.allTrainingData[folderName][i-1]
            img_right = self.allTrainingData[folderName][i]

            try:
                imgL = cv2.imread(self.trainDataPath + folderName + "/RectGrabber/" + img_left)
                imgR = cv2.imread(self.trainDataPath + folderName + "/RectGrabber/" + img_right)
                
                imgL = cv2.resize(imgL, (600,450))
                imgR = cv2.resize(imgR, (600,450))
                print(f"{(i//2) + 1}. Image names: {img_left} & {img_right}\n\t\tShape-L: {imgL.shape}     Shape-R: {imgR.shape}\n")
                
                if not self.showFrames(["Left frame", "Right frame"], [imgL, imgR]):
                    print(f"Couldn't show sample images from showSampleImages function")
            
            except Exception as err:
                print(err)
                if imgL is None:
                    print(f"[!]Couldn't load L-image: {img_left}")
                if imgR is None:
                    print(f"[!]Couldn't load R-image: {img_right}")
                continue
                
        print("---------------------\n[+] Finished viewing initial samples.")


    def frameFromLR(self):
        ### TODO: Will have to refer to the 14th paper later
        pass
    

    def removeNoise(self):
        ### TODO: Future optimization
        pass


    def detectPedestriansAnnotations(self, dirName, imageList):
        '''
            Brief: Making bounding boxes using the annotations provided in dataset
            Need to use SQL queries here to extract info!!! (.db files are present)
        '''
        print("\nCreating Bounding Boxes...")
        print(f"\nDRAWING FOR {dirName}")
        path_ = "/content/drive/MyDrive/Trajectory_Res/Dataset/Dataset_Dailmer/TrainingData_Annotations/"
        path_ += str(dirName)+"/LabelData/"
        # creating file path
        dbfile = path_ + "meas.db"
        
        toRet = []
        with open(dbfile) as f:
            
            rd = f.readlines()
            # Text formatting params:
            font                   = cv2.FONT_HERSHEY_COMPLEX_SMALL
            fontScale              = 0.7
            fontColor              = (255, 255, 255)
            lineType               = 2
            for i in range(len(rd)):
                imgName = ""
                box_cos = ""
                
                if "img" in rd[i]:
                    imgName = rd[i][:-1]
                    
                    if i+6 < len(rd):
                        box_cos = rd[i+6].split()
                        if len(box_cos) != 4:
                            continue
                    else:
                        continue
                    
                    imgName = "imgrect_"+imgName[4:] 
                    #print(f"{imgName}:{box_cos}")
                    
                    img = cv2.imread( "/content/drive/MyDrive/Trajectory_Res/Dataset/Dataset_Dailmer/Data/TrainingData/" 
                                     + dirName + "/RectGrabber/" + imgName )
                    
                    if img is None:
                        print("[-]Image could not load!")
                        continue

                    box_cos = [int(e) for e in box_cos]

                    if dirName not in self.detectedDataBox:
                        self.detectedDataBox[dirName] = []
                        
                    self.detectedDataBox[dirName].append(box_cos) 
                    x,y,a,b = box_cos # Top left: (x,y) ; Bottom Right: (a,b)
                    cv2.rectangle(img, (x, y), (a,b), (0, 255, 0), 2)
                    #cv2.putText(img,'Person', (int(x),int(y)), font, fontScale,fontColor,lineType)
                    #cv2_imshow(img)
                    #cv2.waitKey(0)
                    toRet.append(img)
            
            f.close()
            return toRet

    def saveJPG_for_ViPDL(self, folderName):
        for filee in self.allTrainingData[folderName]:
            fileP = self.trainDataPath + str(folderName) + "/RectGrabber/"
            from pathlib import Path
            gray = Image.open(Path(fileP) / filee)
            gray_img = T.Grayscale(num_output_channels=3)(gray) # convert to 3 channels for ViPDL Panoptic segmentation
            #plt.imshow(np.asarray(gray_img))
            
            if os.path.exists(f'/content/drive/MyDrive/Trajectory_Res/JPGs/{folderName}') == False:
              os.mkdir(f'/content/drive/MyDrive/Trajectory_Res/JPGs/{folderName}')

            new_file = f"/content/drive/MyDrive/Trajectory_Res/JPGs/{folderName}/{filee[:-4]}.jpg"
            cv2.imwrite(new_file, np.asarray(gray_img))
    
    def __init__(self):
        self.trainDataPath = "/content/drive/MyDrive/Trajectory_Res/Dataset/Dataset_Dailmer/Data/TrainingData/"
        self.dirs = []
        self.allTrainingData = dict()
        if self.readData(self.trainDataPath):
            #self.showSampleImages()
            self.detectedData = {}
            self.detectedDataBox = {} # Contains TL&BR co-ordinates of BB box/frame (directory-wise)
            doOneDir = True
            for key in self.allTrainingData:
                if doOneDir:
                    self.detectedData[key] = self.detectPedestriansAnnotations(key, self.allTrainingData[key])
                    #self.saveJPG_for_ViPDL(key)
                    break
                else:
                    self.detectedData[key] = self.detectPedestriansAnnotations(key, self.allTrainingData[key])
                    cv2.destroyAllWindows()

In [ ]:
df = preProcessData()


Creating Bounding Boxes...

DRAWING FOR 2012-10-11_145520


## ViPDL2 panoptic segmentation ✅
---

*Using the standard pre-trained resnet50_beta or ViP deeplab trained on
cityscapes dataset (frozen weights)*



In [ ]:
import collections
import tempfile

from matplotlib import gridspec
import urllib

import tensorflow as tf

In [ ]:
DatasetInfo = collections.namedtuple(
    'DatasetInfo',
    'num_classes, label_divisor, thing_list, colormap, class_names')


def _cityscapes_label_colormap():
  """Creates a label colormap used in CITYSCAPES segmentation benchmark.

  See more about CITYSCAPES dataset at https://www.cityscapes-dataset.com/
  M. Cordts, et al. "The Cityscapes Dataset for Semantic Urban Scene Understanding." CVPR. 2016.

  Returns:
    A 2-D numpy array with each row being mapped RGB color (in uint8 range).
  """
  colormap = np.zeros((256, 3), dtype=np.uint8)
  colormap[0] = [128, 64, 128]
  colormap[1] = [244, 35, 232]
  colormap[2] = [70, 70, 70]
  colormap[3] = [102, 102, 156]
  colormap[4] = [190, 153, 153]
  colormap[5] = [153, 153, 153]
  colormap[6] = [250, 170, 30]
  colormap[7] = [220, 220, 0]
  colormap[8] = [107, 142, 35]
  colormap[9] = [152, 251, 152]
  colormap[10] = [70, 130, 180]
  colormap[11] = [220, 20, 60]
  colormap[12] = [255, 0, 0]
  colormap[13] = [0, 0, 142]
  colormap[14] = [0, 0, 70]
  colormap[15] = [0, 60, 100]
  colormap[16] = [0, 80, 100]
  colormap[17] = [0, 0, 230]
  colormap[18] = [119, 11, 32]
  return colormap


def _cityscapes_class_names():
  return ('road', 'sidewalk', 'building', 'wall', 'fence', 'pole',
          'traffic light', 'traffic sign', 'vegetation', 'terrain', 'sky',
          'person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle',
          'bicycle')


def cityscapes_dataset_information():
  return DatasetInfo(
      num_classes=19,
      label_divisor=1000,
      thing_list=tuple(range(11, 19)),
      colormap=_cityscapes_label_colormap(),
      class_names=_cityscapes_class_names())


def perturb_color(color, noise, used_colors, max_trials=50, random_state=None):
  """Pertrubs the color with some noise.

  If `used_colors` is not None, we will return the color that has
  not appeared before in it.

  Args:
    color: A numpy array with three elements [R, G, B].
    noise: Integer, specifying the amount of perturbing noise (in uint8 range).
    used_colors: A set, used to keep track of used colors.
    max_trials: An integer, maximum trials to generate random color.
    random_state: An optional np.random.RandomState. If passed, will be used to
      generate random numbers.

  Returns:
    A perturbed color that has not appeared in used_colors.
  """
  if random_state is None:
    random_state = np.random

  for _ in range(max_trials):
    random_color = color + random_state.randint(
        low=-noise, high=noise + 1, size=3)
    random_color = np.clip(random_color, 0, 255)

    if tuple(random_color) not in used_colors:
      used_colors.add(tuple(random_color))
      return random_color

  print('Max trial reached and duplicate color will be used. Please consider '
        'increase noise in `perturb_color()`.')
  return random_color


def color_panoptic_map(panoptic_prediction, dataset_info, perturb_noise):
  """Helper method to colorize output panoptic map.

  Args:
    panoptic_prediction: A 2D numpy array, panoptic prediction from deeplab
      model.
    dataset_info: A DatasetInfo object, dataset associated to the model.
    perturb_noise: Integer, the amount of noise (in uint8 range) added to each
      instance of the same semantic class.

  Returns:
    colored_panoptic_map: A 3D numpy array with last dimension of 3, colored
      panoptic prediction map.
    used_colors: A dictionary mapping semantic_ids to a set of colors used
      in `colored_panoptic_map`.
  """
  if panoptic_prediction.ndim != 2:
    raise ValueError('Expect 2-D panoptic prediction. Got {}'.format(
        panoptic_prediction.shape))

  semantic_map = panoptic_prediction // dataset_info.label_divisor
  instance_map = panoptic_prediction % dataset_info.label_divisor
  height, width = panoptic_prediction.shape
  colored_panoptic_map = np.zeros((height, width, 3), dtype=np.uint8)

  used_colors = collections.defaultdict(set)
  # Use a fixed seed to reproduce the same visualization.
  random_state = np.random.RandomState(0)

  unique_semantic_ids = np.unique(semantic_map)
  for semantic_id in unique_semantic_ids:
    semantic_mask = semantic_map == semantic_id
    if semantic_id in dataset_info.thing_list:
      # For `thing` class, we will add a small amount of random noise to its
      # correspondingly predefined semantic segmentation colormap.
      unique_instance_ids = np.unique(instance_map[semantic_mask])
      for instance_id in unique_instance_ids:
        instance_mask = np.logical_and(semantic_mask,
                                       instance_map == instance_id)
        random_color = perturb_color(
            dataset_info.colormap[semantic_id],
            perturb_noise,
            used_colors[semantic_id],
            random_state=random_state)
        colored_panoptic_map[instance_mask] = random_color
    else:
      # For `stuff` class, we use the defined semantic color.
      colored_panoptic_map[semantic_mask] = dataset_info.colormap[semantic_id]
      used_colors[semantic_id].add(tuple(dataset_info.colormap[semantic_id]))
  return colored_panoptic_map, used_colors


def vis_segmentation(image,
                     panoptic_prediction,
                     dataset_info,
                     perturb_noise=60):
  """Visualizes input image, segmentation map and overlay view."""
  plt.figure(figsize=(30, 20))
  grid_spec = gridspec.GridSpec(2, 2)

  ax = plt.subplot(grid_spec[0])
  plt.imshow(image)
  plt.axis('off')
  ax.set_title('input image', fontsize=20)

  ax = plt.subplot(grid_spec[1])
  panoptic_map, used_colors = color_panoptic_map(panoptic_prediction,
                                                 dataset_info, perturb_noise)
  plt.imshow(panoptic_map)
  plt.axis('off')
  ax.set_title('panoptic map', fontsize=20)

  ax = plt.subplot(grid_spec[2])
  plt.imshow(image)
  plt.imshow(panoptic_map, alpha=0.7)
  plt.axis('off')
  ax.set_title('panoptic overlay', fontsize=20)

  ax = plt.subplot(grid_spec[3])
  max_num_instances = max(len(color) for color in used_colors.values())
  # RGBA image as legend.
  legend = np.zeros((len(used_colors), max_num_instances, 4), dtype=np.uint8)
  class_names = []
  for i, semantic_id in enumerate(sorted(used_colors)):
    legend[i, :len(used_colors[semantic_id]), :3] = np.array(
        list(used_colors[semantic_id]))
    legend[i, :len(used_colors[semantic_id]), 3] = 255
    if semantic_id < dataset_info.num_classes:
      class_names.append(dataset_info.class_names[semantic_id])
    else:
      class_names.append('ignore')

  plt.imshow(legend, interpolation='nearest')
  ax.yaxis.tick_left()
  plt.yticks(range(len(legend)), class_names, fontsize=15)
  plt.xticks([], [])
  ax.tick_params(width=0.0, grid_linewidth=0.0)
  plt.grid('off')
  plt.show()

In [ ]:
MODEL_NAME = 'resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model'  # @param ['resnet50_os32_panoptic_deeplab_cityscapes_crowd_trainfine_saved_model', 'resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model', 'wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model', 'swidernet_sac_1_1_1_os16_panoptic_deeplab_cityscapes_trainfine_saved_model', 'swidernet_sac_1_1_3_os16_panoptic_deeplab_cityscapes_trainfine_saved_model', 'swidernet_sac_1_1_4.5_os16_panoptic_deeplab_cityscapes_trainfine_saved_model', 'axial_swidernet_1_1_1_os16_axial_deeplab_cityscapes_trainfine_saved_model', 'axial_swidernet_1_1_3_os16_axial_deeplab_cityscapes_trainfine_saved_model', 'axial_swidernet_1_1_4.5_os16_axial_deeplab_cityscapes_trainfine_saved_model', 'max_deeplab_s_backbone_os16_axial_deeplab_cityscapes_trainfine_saved_model', 'max_deeplab_l_backbone_os16_axial_deeplab_cityscapes_trainfine_saved_model']


_MODELS = ('resnet50_os32_panoptic_deeplab_cityscapes_crowd_trainfine_saved_model',
           'resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'wide_resnet41_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'swidernet_sac_1_1_1_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'swidernet_sac_1_1_3_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'swidernet_sac_1_1_4.5_os16_panoptic_deeplab_cityscapes_trainfine_saved_model',
           'axial_swidernet_1_1_1_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'axial_swidernet_1_1_3_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'axial_swidernet_1_1_4.5_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'max_deeplab_s_backbone_os16_axial_deeplab_cityscapes_trainfine_saved_model',
           'max_deeplab_l_backbone_os16_axial_deeplab_cityscapes_trainfine_saved_model')
_DOWNLOAD_URL_PATTERN = 'https://storage.googleapis.com/gresearch/tf-deeplab/saved_model/%s.tar.gz'

_MODEL_NAME_TO_URL_AND_DATASET = {
    model: (_DOWNLOAD_URL_PATTERN % model, cityscapes_dataset_information())
    for model in _MODELS
}

MODEL_URL, DATASET_INFO = _MODEL_NAME_TO_URL_AND_DATASET[MODEL_NAME]

In [ ]:
model_dir = tempfile.mkdtemp()

download_path = os.path.join(model_dir, MODEL_NAME + '.gz')
urllib.request.urlretrieve(MODEL_URL, download_path)

!tar -xzvf {download_path} -C {model_dir}

LOADED_MODEL = tf.saved_model.load(os.path.join(model_dir, MODEL_NAME))

resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model/
resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model/assets/
resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model/saved_model.pb
resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model/variables/
resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model/variables/variables.data-00000-of-00001
resnet50_beta_os32_panoptic_deeplab_cityscapes_trainfine_saved_model/variables/variables.index


# Stream Data In & Out from JPGs dir
---
Future:
*   Pre-compute all environment maps. (ViPDL2 fps: 5) OR
*   **Replace Segmentation algorithm with a faster one and make everything real-time**





In [ ]:
def segment_Image(imgPath = ''):
  UPLOADED_FILE = imgPath
  with tf.io.gfile.GFile(UPLOADED_FILE, 'rb') as f:
    im = np.array(Image.open(f))

  output = LOADED_MODEL(tf.cast(im, tf.uint8))
  #vis_segmentation(im, output['panoptic_pred'][0], DATASET_INFO)
  return output['semantic_probs'][0] # This is THETA -> ENV_MAP


In [ ]:
env_map = segment_Image("/content/imgrect_000000163_c0.jpg")

# Segmentation Done! ⭐
---
# Moving on to the main Prediction Model

In [ ]:
class model:
    '''
    Brief:
        Goal Directed Pedestrian Prediction 
        (A markovian approach for pedestrian trajectory prediction)
        
    Receives:
        Pre-processed and pedestrain detected data from preProcessData class
        
    Returns:
        Predicted Trajectory of each pedestrian in scene
                
    '''
    
    # State Transition stuff ahead:
    def compute_X(self, dirName = '2012-04-02_115542', numFramesToObserve = 30):
        # COMPUTING ONLY FOR THE TL PIXEL OF RECTANGLE 
        for i in range(1, numFramesToObserve):
            x1, y1, a1, b1 = self.dfBox[dirName][i-1]
            x2, y2, a2, b2 = self.dfBox[dirName][i]
            vx = (x2 - x1) * self.fps    # TODO: Use more valid points to compute an average velocity
            vy = (y2 - y1) * self.fps
            #print(f"*** Frames {i-1} and {i} ***\nvx: {vx} px/sec\nvy: {vy} px/sec")
            psi = 0
            if (x2-x1) != 0:
                psi = math.atan2(  (y2-y1) , (x2-x1) ) # Slope 
            #print(f"phi_t: {math.degrees(psi)} degrees\n")
            self.uni.append([vx, vy, 0])
            self.X.append([x1 + vx/self.fps, y1 + vy/self.fps, 0 + psi]) # Xt = Xt-1 + u(v_t, psi_t) 
        self.eq3_dist_X()
        #self.get_XTs()
    
    def eq3_dist_X(self):
        '''
            phi_t = p( X_t | X_t-1 )
            p(Xt|Xt−1) = p(Xt−1) ⊗ p(u(vt, ψt))
        '''
        self.distX = np.zeros(shape=(640, 1176))
        n = len(self.X)
        
        for i in range(n):
            self.distX[int(self.X[i][1])][int(self.X[i][0])] += 1 / n
        
        #print(self.distX)
        #print(self.distUni)
        
        #plt.subplot(1,2,1)
        #plt.imshow(self.distX)
        #plt.subplot(1,2,2)
        #plt.imshow(self.distUni)
        #plt.show()
        # Convolve the two grid distributions
        #self.Xt_given_Xt_1 = conv_pmf = signal.fftconvolve(self.distX, self.distUni, 'same')
        
        #print(f"P(Xt|Xt-1) = P(Xt-1) * p(u(vt,psit)) =\n{self.Xt_given_Xt_1}")
        
        #for i in range(640):
        #    for j in range(1176):           
        #        if self.Xt_given_Xt_1[i][j] >= 1:
        #            print(f"cell({i},{j})'s p(Xt|Xt-1) = {self.Xt_given_Xt_1[i][j]}\nNOT POSSIBLE! RE-CHECK CONVOLUTION")
                
        #plt.imshow(disp)
        #plt.show()
        self.eq4()
    
    def discretization_2_grid(self, p, x, y):
        '''
            Consider 1x1 cells -> assign probability(from incremental distribution) to each cell
            At the end: A (the convolution filter mask) is obtained 
        '''
        self.A[int(y)][int(x)] = p
        
    
    
    def mean_var(self, samples):
        mean = sum(samples) / len(samples)
        var = 0
        for e in samples:
            var += (e - mean)**2
        var /= len(samples)-1 
        return mean, var
    
    
    def vonMises(self, samples):
        R = 0                      # 1/N * Summation{e^i.\theta}
        for i in range(len(samples)):
            R += math.cos((i+1)*samples[i]) + 1j*math.sin((i+1)*samples[i])
        
        Rbar = np.abs(R)/len(samples)
        kappa_v = (2*Rbar - (Rbar**3)) / (1 - (Rbar**2)) # This is an approximation for fast computation only!
        
        # TODO: Find closer approximation from Bessel function or Ap^-1
        return Rbar, kappa_v
    
    
    def eq4(self):
        '''
        Model the incremental distribution:
            This is p(u(v, psi)) effectively...
            
            p(∆x, ∆y, ∆ψ) ∝  exp(−(∆x−∆tv cos(ψ))^2/2σ_v^2)
                            .exp(−(∆y−∆tv sin(ψ))^2/2σ_v^2)
                            .exp(κ∆ψ cos(∆ψ))
                            .exp(κv cos(∠(∆y, ∆x) − ψ))
                            
        Then call self.discretization_2_grid() to get A (the convolution filter mask)
        Then call eq5 to get phi_t using A and phi_t-1
        
        v   -> Normal Dist.             
        psi -> circular Normal Dist.
        '''
        # p[x][y][psi] = model it!!!
        vx_mu, vx_var = self.mean_var([e[0] for e in self.uni])
        vy_mu, vy_var = self.mean_var([e[1] for e in self.uni])
        psi_mu, psi_kappa = self.vonMises([e[2] for e in self.X])
        
        kappa_nonAligned = 1   # TODO: Figure out if this can be found from the data or is a free parameter
        
        # TODO: Now find p(del_x, del_y, del_psi)
        x,y,z = self.X[-1] # Get filter: A, from last observed position-orientation tuple
        # Keeping to a region of -20 to +20
        for i in range(-50, 51):
            for j in range(-50, 51):
                xprev = x + i
                yprev = y + j
                if i != 0:
                    zprev = z - math.atan2((y - yprev),(x - xprev))
                else:
                    if y - yprev < 0:
                        zprev = z + math.pi/2
                    else:
                        zprev = z - math.pi/2
        
                # Constant of proportionality  
                from scipy.special import kn
                k = 8*(math.pi**3)*math.sqrt(vx_var)*math.sqrt(vy_var)*kn(0, psi_kappa)*kn(0, kappa_nonAligned) 
                
                P_del_xyz = 1/k
                P_del_xyz *= math.exp(-(((x-xprev) - vx_mu/self.fps)**2)/(2*vx_var)) 
                #print(self.P_delDist, math.exp(-(((x-xprev) - vx_mu/self.fps)**2)/(2*vx_var)) )
                P_del_xyz *= math.exp(-(((y-yprev) - vy_mu/self.fps)**2)/(2*vy_var)) 
                #print(self.P_delDist,  math.exp(-(((y-yprev) - vy_mu/self.fps)**2)/(2*vy_var)))
                
                P_del_xyz *= math.exp(psi_kappa * math.cos(z - zprev))
                #print(self.P_delDist, psi_kappa )
                
                angle = 0 # goes in non-alaigned term
                if  x-xprev == 0:
                    angle = math.pi/2 # arctan(infinity) is pi/2
                    if y - yprev < 0:
                        angle *= -1 # arctan(-inf) is -pi/2
                else:    
                    angle = math.atan2((y-yprev),(x-xprev))
                
                P_del_xyz *= math.exp( kappa_nonAligned * math.cos(angle - z) )
                #print(P_del_xyz,  math.exp( kappa_nonAligned * math.cos(angle - z)))
                self.discretization_2_grid(P_del_xyz, x+i, y+j)                      
        print(f"Filter A computed!")
        
        # Normalize to be double sure! 
        xmax, xmin = self.A.max(), self.A.min()
        self.A = (self.A - xmin)/(xmax - xmin)
        
        plt.figure(figsize = (16,12))
        plt.title("Filter A")
        plt.imshow(self.A, cmap='hot', interpolation='nearest')
        plt.show()
        self.eq5()
        
    
    def eq5(self):
        '''
            Φt ∝ A ⊗ Φt−1
            where Φt = P( Xt | Xt-1 ) {contained in self.Xt_given_Xt_1} 
            Convolve here
            
            Do something like:
            #self.Xt_given_Xt_1 = conv_pmf = signal.fftconvolve(self.distX, self.A, 'same')
        '''
        self.Xt_given_Xt_1 = signal.fftconvolve(self.distX, self.A, 'same')
        self.get_XTs()
        
    
    def get_XTs(self): # Adaptive? Monte Carlo Simulation
        if self.XT == None: # Uniform sampling 
            self.XT = dict()
            centerX, centerY, z = self.X[-1]
            # Create 100 particles at a distance of 100 pixels uniformly distributed
            self.XT[0] = []
            for i in range(500):
                r = random.randint(10,400) * math.sqrt(random.random())
                theta = random.random() * 2 * math.pi
                x = centerX + r * math.cos(theta)
                y = 640 - centerY + r * math.sin(theta)
                if x <= 640 and y <= 1176 and y >= 0 and x >= 0:
                  self.XT[0].append((int(x),int(y)))
        else: # Resample according to previous distribution
            # GMM Stuff
            pass
        self.eq7()
    
    
    def eq7(self):
        '''
            Carry out forward and backward steps to get each p(Xt | X0, XT) acc. to:
            p(Xt|X0, XT ) ∝ Φ+t Φ−t
        '''
        predStep = self.eq8() * self.eq9(phi_minus_t)
    
    
    # Location Prior stuff also needed for eq8 and 9
    def eq8(self):
        '''
            Forward step:
            Φ+t ∝ p(Xt|Θt)(A ⊗ Φ+t−1)
            
            where Θt is the online map
                  p(X|θi) is the location prior (call eq10 here)
        '''
        phi_plus_t = self.Xt_given_Xt_1 * self.eq10() # Forward step prediction: Done
        return phi_plus_t
    
    def eq9(self, phi_minus_t):
        '''
            Backward step:
            Φ−t−1 ∝ p(Xt|Θt)(A^(−1) ⊗ Φ−t)
            
            call eq10 from here
        '''
        phi_minus_tM1 = self.eq10() * signal.fftconvolve(np.linalg.inv(self.A), phi_minus_t, 'same')
        return phi_minus_tM1
    
    def SLP(self, weights, x, whatPhase):
      '''
        x -> semantic map (640, 1176, 19)
      '''
      def forwardpass(self, x, weights):
        # output = 1 / 1 + exp(-weights^T(1x19) * input vec(19x1))
        return 1 / 1 + math.exp(-self.weights.T @ x)

      def activation(self, num):
        return num

      def update(self, x, ytrue):
        err = ytrue - activation(self.forwardpass(x))
        self.weights = self.weights + learningRate * err * x
        return abs(err)

      def loss(self, iter):
        if iter % 100 == 0:
          print(f"epoch {int(iter/100 + 1)}/{int(len(trainY)/100)} ======> Absolute Loss = {round(self.lossVal/(iter+1), 3)}")

      if whatPhase is 'train':
        trainY = []
        ytrue = []
        learningRate = 0.5
        pass
      else:
        return activation(forwardpass(x, weights))
    
    def eq10(self, imgPath):
        '''
            Computes Location prior: p(X|θi) acc to:
            
            Single Layer Perceptron: 
            p(X|θi) = 1 / 1 + exp (−a^T . θ_i)
            
            where a^T represents weighting params for different features. 
            
            Labels:
            'road', 'sidewalk', 'building', 'wall', 'fence', 'pole',
            'traffic light', 'traffic sign', 'vegetation', 'terrain', 'sky',
            'person', 'rider', 'car', 'truck', 'bus', 'train', 'motorcycle',
            'bicycle'

        '''
        THETA = segment_Image()
        a = np.zeros((19,1))
        a[0] = 0.45
        a[1] = 0.42
        a[9] = 0.1
        a[15] = 0.03
        self.SLP(a, 'train')
        locationPrior = np.zeros((640, 1176))
        for i in range(640):
          for j in range(1176):
            locationPrior[i][j] = self.SLP(torch.from_numpy(a), THETA[i][j], 'predict')
        
        return locationPrior # NOTE: All cells are independent
    
    
    def eq11(self):
        '''
            Path Distribution: 
            p(XM|X0, XT , Θt) = 1 −PI[(M) <-- (t˜=0)](1 − p(Xt˜|X0, XT , Θt))
        '''
        pass
    
    
    def eq12(self):
        '''
            Cost function to get best weighting params, i.e, a:
            J(a) = −Summation[ ζi ∈ Xj ] Summation[ Xj ∈ ζi ] log(p(X = Xj |X0, XT , Θt))
        '''
        pass
    
    # Goal Distribution XT Estimation ahead
    def eq13(self):
        '''
            p(Xt) ::= estimated distribution for the pedestrian’s state at time = t. 
            p(X−t|X0, XT , Θt) ::= the distribution of that state obtained from prediction at t-1. 
        
            [!] After Assumption: Independence
            Can obtain:
        
                pX− (X−t|X0, XT , Θt) ∝ pX− (X0, XT , Θt|X−t)p(X−t)
        
        '''
        pass
    
    
    def eq14(self):
        '''
            p(XT ) ∝ Integral of pX− (X0, XT , Θt|Xt).p(Xt).dX
            
            [+] Evaluated for the individual particles and the result is used for reweighting
            Next TODO: Unlikely particles are discarded and randomly resampled at other locations.
        '''
        pass
    
    
    def __init__(self, bb_Images, bb_cos):
        self.df = bb_Images
        self.dfBox = bb_cos
        self.grid = np.zeros(shape=(640, 1176)) # Grid
        self.fps = 30                          # fps
        self.X = []                            # Contains last t positions & orientations: (x_t, y_t, psi_t)
        self.distX = None                      # Contains X's distribution on grid
        self.uni = []                          # Contains unicycle vector of motion: u(vt, psi_t) 
        self.distUni = None                    # Contains uni's distribution on grid as per X's co-ordinates
        self.Xt_given_Xt_1 = None              # Contains distribution: P(X_t | X_t-1)
        self.px0 = 0
        self.XT = None                         # Latent Variable: Gaussian Mixture Model + Particle Filter
        self.A = np.zeros(shape=(640, 1176))
        for key in self.df:
            self.compute_X(key)  # Watch 1 sec to estimate X

## DB1: First 30 frames do not convey all the information.
## DB2: 